In [12]:
import pandas as pd
import os
from google.cloud import storage

file_prefix = "right_left_up_down_1200"
data_folder = "data_" + file_prefix
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcsaccesskey.json"

def download_files_to_folder():
    client = storage.Client()

    bucket = "something-somethingv2-zips"
    bucket = client.bucket(bucket)

    print(list(bucket.list_blobs()))

    for blob in bucket.list_blobs():
        blob.download_to_filename(blob.name)



download_files_to_folder()

[<Blob: something-somethingv2-zips, 20bn-something-something-v2-00.zip, 1700606448445788>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-01.zip, 1700606448551130>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-02.zip, 1700606449029072>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-03.zip, 1700608686951638>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-04.zip, 1700608686938056>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-05.zip, 1700608685625779>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-06.zip, 1700610612904644>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-07.zip, 1700610613357866>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-08.zip, 1700610609076515>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-09.zip, 1700611201322953>, <Blob: something-somethingv2-zips, 20bn-something-something-v2-10.zip, 1700643640453112>, <Blob: so

# Fetch Something Something Data-set

[Something Something v2 webpage](https://developer.qualcomm.com/software/ai-datasets/something-something)

The video data is provided as one large TGZ archive, split into parts of 1 GB maximum. The total download size is 19.4 GB. The archive contains webm-files, using the VP9 codec, with a height of 240px. Files are numbered from 1 to 220847.

For each video in the training and validation sets there is an object annotation in addition to the video label, if applicable. For example, for a label like "Putting [something] onto [something]," there is also an annotated version, such as "Putting a cup onto a table." In total, there are 318,572 annotations involving 30,408 unique objects.


[download instructions](./links/20bn-something-something_download_instructions_-_091622.pdf)

In [3]:
import os
import subprocess

os.mkdir("./data/")
os.mkdir("./data/extracted")

In [9]:
zip_files = [file for file in os.listdir('./') if '.zip' in file]

In [ ]:
for file in zip_files:
    subprocess.call(
        f'unzip {file} -d ./data/extracted',
        shell=True
    )